In [1]:
import sys
import os

sys.path.insert(1,'/data1/gridsan/groups/manthiram_lab/Utils')
sys.path.insert(1,'/data1/gridsan/groups/manthiram_lab/DFT/DFT_script_gen')
import utils
from utils import make_xyz_from_output,checkvibs, check, no_to_symbol
import convertxyz
from convertxyz import mk_xyz_from_smiles_string, get_xyz_from_mol, write_xyz_from_xyz_arr,combine_labels_and_xyz
import numpy as np
import pandas as pd
import pickle
import re
import importlib
import cclib
from os.path import exists
import re
import subprocess
import matplotlib.pyplot as plt
from tqdm import tqdm
import matplotlib.pyplot as plt
import plot as pl
import pathlib
from datetime import datetime, timedelta
import pickle as pkl

from rdkit import Chem

In [2]:
importlib.reload(pl)
importlib.reload(utils)
importlib.reload(convertxyz)

<module 'convertxyz' from '/data1/gridsan/groups/manthiram_lab/DFT/DFT_script_gen/convertxyz.py'>

In [3]:
#Define Current directory
cwd= '/home/gridsan/jmaalouf/vHTP/Code/ylide/Library'

# Energy Functions

In [4]:
def calcE(G_ox,G_red,n=1):
    """ 
    Calculations the redox potential of a reaction using dG=-nFE-4.28 to calculate vs SHE.
    4.28 is SHE vs vacuum.

    Arguments:
    G_ox (int): Gibbs free energy of oxidized species in hartree references to vacuum.
    G_red (int): Gibbs free energy of reduced species in hartree references to vacuum.

    Returns:
    E vs SHE (int): Redox potential in volts references vs SHE. 

    """  
    F=96485 #Faraday's constant [=] C/mol
    dG=G_ox-G_red
    E= dG*2625.5*1000/(n*F) - 4.28 #Volts
    return E




def calcDPFE(G_ylide_h,G_ylide,solvent='gas'):
    """ 
    Calculations the deprotonation Free Energy of a reaction, specifically going from ylide_h to ylide

    Arguments:
    ylide_h (int): Gibbs free energy of protonated ylide in hartree references to vacuum.
    ylide (int): Gibbs free energy of ylide in hartree references to vacuum.

    Returns:
    DPFE (int): Deprotonation Free Energy in kJ/mol. 

    """ 
    
    #Dictionaory of H+ energies at the M062X 
    proton_dict={
    'gas': -0.010000,
    'acetonitrile': -0.223572
    }
    
    possible_solvents=['gas','acetonitrile']
    assert solvent in possible_solvents, "VALID SOLVENT NOT SPECIFIED"
    
    return -(G_ylide_h-G_ylide-proton_dict[solvent])*2625.5    


def calcGHBind(G_ylide_h,G_ylide_rad,solvent='gas'):
    """ 
    Calculations the Hydrogen atom Binding Energy of a reaction, specifically going from ylide_rad to ylide_h

    Arguments:
    ylide_h (int): Gibbs free energy of protonated ylide in hartree references to vacuum.
    ylide_rad (int): Gibbs free energy of ylide radical in hartree references to vacuum.

    Returns:
    GHBind (int): Hydrogen Atom Binding energy of an ylide in kJ/mol. 

    """ 
    
    #Dictionaory of H atom energies at the M062X 
    Hatom_dict={
    'gas': -0.508484,
    'acetonitrile': -0.508404
    }
    
    H2_dict={
    'gas': -1.169741,
    'acetonitrile':-1.169345
    }
    
    possible_solvents=['gas','acetonitrile']
    
    assert solvent in possible_solvents, "VALID SOLVENT NOT SPECIFIED"
    
    return (G_ylide_h-G_ylide_rad-H2_dict[solvent]/2)*2625.5

In [3]:
df=pd.read_csv('../sheets/Ylides_YlideRads_Cleaned.csv')
y=df['Ylides'].to_list()
yr=df['Ylides Rad'].to_list()
yh=df['Ylides H'].to_list()

## Get Num Atoms

In [6]:
num_atoms=[]
num_atoms_total=[]
for i, s in enumerate(tqdm(y)):
    mol=Chem.MolFromSmiles(s)
    num_atoms.append(mol.GetNumAtoms())
    mol=Chem.AddHs(mol)
    num_atoms_total.append(mol.GetNumAtoms())

100%|██████████| 19097/19097 [00:01<00:00, 9561.11it/s]


In [8]:
with open(f"../pickled_data/num_atoms.pkl", "wb") as file: 
    pickle.dump(num_atoms, file)
    
with open(f"../pickled_data/num_atoms_total.pkl", "wb") as file: 
    pickle.dump(num_atoms_total, file)

# Check for Failed Calcs

In [10]:
trueChoices = ['true', 't','T']
falseChoices = ['false', 'f','F']

inp=input("Do you want to resubmit the memory allocation errors? (T/F)").lower()

assert inp in trueChoices or inp in falseChoices
if inp in trueChoices:
    resubmit=True
elif inp in falseChoices:
    resubmit=False

orig_dir='/home/gridsan/groups/manthiram_lab/vHTP/Code/ylide/Library'

#failed_dict={}
#geom_failed_dict={}

functional='M062X'
basis= 'Def2SVPD'
solvorgas='solv'
solvmethod='SMD'
solvent='acetonitrile'

if solvorgas== 'gas':
    ylide_type=[['ylide',''],['ylide_rad','_y_h_opt'],['ylide_h','']]
elif solvorgas =='solv':
    ylide_type=[['ylide','_gas_preopt'],['ylide_rad','_gas_preopt'],['ylide_h','_gas_preopt']]
    ylide_type=[['ylide_h','_gas_preopt']]

start=0
stop=10000
n=stop-start
error_occurred=False

for v,y_list in enumerate(ylide_type):
    
    y=y_list[0]
    sub_suff=y_list[1]
    
    failed_dict[f'{y}{sub_suff}']=[]
    geom_failed_dict[f'{y}{sub_suff}']=[]
    
    for i in tqdm(range(start,stop)):
        n_str=str(i).zfill(7)
        
        
        assert solvorgas == 'gas' or solvorgas=='solv','SOLVORGAS NOT SET TO A VALID VALUE'
        
        if solvorgas=='gas':
            path=pathlib.Path(os.path.join(orig_dir, 'Calcs', str(i).zfill(7) , y, functional, basis,'gas', f'{str(i).zfill(7)}_{y}{sub_suff}.log'))
            
        elif solvorgas=='solv':
            path=pathlib.Path(os.path.join(orig_dir, 'Calcs',str(i).zfill(7) , y, functional, basis,solvent,solvmethod, f'{str(i).zfill(7)}_{y}{sub_suff}.log'))
        
        par_path=path.parent.absolute()
        
        if exists(path):
            
            if check(path, 'Normal termination of Gaussian', count=2)==False:
                error_occurred=True
                
                if check(str(path),'galloc:  could not allocate memory.'):
                    
                    #print(f'MEMRORY ALLOCATION ERROR WITH {y}{sub_suff} MOLECULE {i}')
                    failed_dict[f'{y}{sub_suff}'].append(i)
                    
                    if resubmit:
                        os.chdir(par_path)
                        subprocess.run(f'sbatch {n_str}_{y}{sub_suff}.sh',shell=True)
                        os.chdir(orig_dir)
                                
                elif check(str(path),'Converged',120):
                    #print(f'GEOMETRY OPT ERROR WITH {y}{sub_suff} MOLECULE {i}')
                    #failed_dict[f'{y}{sub_suff}'].append(i)
                    geom_failed_dict[f'{y}{sub_suff}'].append(i)
                    
                else:
                    #Try one more time to extract the data because there could still be some optimized geometry
                    #steps that can be extracted
                    #print(f'UNKNOWN ERROR, {y}{sub_suff} MOLECULE {i} CALCULATION DID NOT FINSIH')
                    failed_dict[f'{y}{sub_suff}'].append(i)
                    
                    if resubmit:
                        os.chdir(par_path)
                        #subprocess.run(f'sbatch {n_str}_{y}{sub_suff}.sh',shell=True)
                        os.chdir(orig_dir)
        else:
            #print(f'{y} MOLECULE {i}{sub_suff} HAS NOT BEEN SUBMITTED')
            failed_dict[f'{y}{sub_suff}'].append(i)
            
            if resubmit:
                os.chdir(par_path)
                subprocess.run(f'sbatch {n_str}_{y}{sub_suff}.sh',shell=True)
                os.chdir(orig_dir)
                
    if error_occurred==False:
        print(f'NO ERRORS WITH {y}{sub_suff}')
        
        
    error_occurred=False
    print('')
    


Do you want to resubmit the memory allocation errors? (T/F) f


  0%|          | 32/10000 [00:00<01:07, 148.45it/s]

MEMRORY ALLOCATION ERROR WITH ylide_h_gas_preopt MOLECULE 3
MEMRORY ALLOCATION ERROR WITH ylide_h_gas_preopt MOLECULE 4
MEMRORY ALLOCATION ERROR WITH ylide_h_gas_preopt MOLECULE 8
MEMRORY ALLOCATION ERROR WITH ylide_h_gas_preopt MOLECULE 13
MEMRORY ALLOCATION ERROR WITH ylide_h_gas_preopt MOLECULE 15
MEMRORY ALLOCATION ERROR WITH ylide_h_gas_preopt MOLECULE 19
MEMRORY ALLOCATION ERROR WITH ylide_h_gas_preopt MOLECULE 21
MEMRORY ALLOCATION ERROR WITH ylide_h_gas_preopt MOLECULE 23
MEMRORY ALLOCATION ERROR WITH ylide_h_gas_preopt MOLECULE 25


  1%|          | 61/10000 [00:00<01:13, 134.45it/s]

MEMRORY ALLOCATION ERROR WITH ylide_h_gas_preopt MOLECULE 34
MEMRORY ALLOCATION ERROR WITH ylide_h_gas_preopt MOLECULE 39
MEMRORY ALLOCATION ERROR WITH ylide_h_gas_preopt MOLECULE 43
MEMRORY ALLOCATION ERROR WITH ylide_h_gas_preopt MOLECULE 49
MEMRORY ALLOCATION ERROR WITH ylide_h_gas_preopt MOLECULE 51
MEMRORY ALLOCATION ERROR WITH ylide_h_gas_preopt MOLECULE 55
MEMRORY ALLOCATION ERROR WITH ylide_h_gas_preopt MOLECULE 57


...

 77%|███████▋  | 7663/10000 [01:44<00:21, 110.56it/s]

MEMRORY ALLOCATION ERROR WITH ylide_h_gas_preopt MOLECULE 7650
MEMRORY ALLOCATION ERROR WITH ylide_h_gas_preopt MOLECULE 7651
MEMRORY ALLOCATION ERROR WITH ylide_h_gas_preopt MOLECULE 7655
MEMRORY ALLOCATION ERROR WITH ylide_h_gas_preopt MOLECULE 7656
MEMRORY ALLOCATION ERROR WITH ylide_h_gas_preopt MOLECULE 7662
MEMRORY ALLOCATION ERROR WITH ylide_h_gas_preopt MOLECULE 7663
MEMRORY ALLOCATION ERROR WITH ylide_h_gas_preopt MOLECULE 7664


 77%|███████▋  | 7676/10000 [01:45<00:32, 72.48it/s] 

MEMRORY ALLOCATION ERROR WITH ylide_h_gas_preopt MOLECULE 7665
MEMRORY ALLOCATION ERROR WITH ylide_h_gas_preopt MOLECULE 7668
MEMRORY ALLOCATION ERROR WITH ylide_h_gas_preopt MOLECULE 7670
MEMRORY ALLOCATION ERROR WITH ylide_h_gas_preopt MOLECULE 7672
MEMRORY ALLOCATION ERROR WITH ylide_h_gas_preopt MOLECULE 7674


100%|██████████| 10000/10000 [02:16<00:00, 73.29it/s]

In [11]:
len(failed_dict['ylide_rad_y_h_opt'])

0

In [17]:
geom_failed_dict['ylide_rad_y_h_opt']

[118, 264]

In [18]:
failed_dict['ylide_rad_y_h_opt']

[118, 262, 264, 266, 324, 480, 485]

In [13]:
with open(f"/data1/groups/manthiram_lab/vHTP/Code/ylide/Library/python_scripts/dicts/failed_dict_{solvorgas}_{'0801'}.pkl", "wb") as file: 
    pickle.dump(failed_dict, file)

In [9]:
#os.chdir(cwd)
with open(f"/data1/groups/manthiram_lab/vHTP/Code/ylide/Library/python_scripts/dicts/failed_dict_{solvorgas}_{'0626'}.pkl", "wb") as file: 
    pickle.dump(failed_dict, file)

In [ ]:
with open(f"../pickled_data/failed_dict_{solvorgas}.pkl", "wb") as file: 
    pickle.dump(failed_dict, file)

In [22]:
with open(f"../pickled_data/geom_failed_dict_{solvorgas}.pkl", "wb") as file: 
    pickle.dump(geom_failed_dict, file)

In [11]:
for key, value in failed_dict.items():
    print(f'{key} has {len(value)} failures and a fail percentage of {len(value)/n*100}%')

ylide_gas_preopt has 269 failures and a fail percentage of 2.69%
ylide_h_gas_preopt has 8638 failures and a fail percentage of 86.38%


In [12]:
for key, value in geom_failed_dict.items():
    print(f'{key} has {len(value)} failures and a fail percentage of {len(value)/n*100}%')

ylide_gas_preopt has 122 failures and a fail percentage of 1.22%
ylide_h_gas_preopt has 44 failures and a fail percentage of 0.44%


# Extract Data

In [27]:
functional='M062X'
basis= 'Def2TZVP'
solvorgas='gas'
solvmethod='SMD'


orig_dir='/home/gridsan/groups/manthiram_lab/vHTP/Code/ylide/Library'
if solvorgas=='gas':
    solvent='gas'
    ylide_type=[['ylide',''],['ylide_rad','_y_h_opt'],['ylide_h','']]
    #ylide_type=[['ylide_h','']]
else:
    solvent='acetonitrile'
    ylide_type=[['ylide','_gas_preopt'],['ylide_rad','_gas_preopt'],['ylide_h','_gas_preopt'] ]

start=1335
stop=1336
n=stop-start
data=np.empty((n,10))
atom_coords=[] #Dim1= Ylide Dim2=Ylide Rad Dim3= Ylide_H
atom_labels=[] #Dim1= Ylide Dim2=Ylide Rad Dim3= Ylide_H
wall_time=[] #Calculation time in seconds

for v,y_list in enumerate(ylide_type):
    
    y=y_list[0]
    sub_suff=y_list[1]
    
    temp_atom_coords=[]
    temp_atom_labels=[]
    temp_wall_time=[]
    
    for k,i in enumerate(tqdm(range(start,stop))):
        n_str=str(i).zfill(7)

        
        
        assert solvorgas == 'gas' or solvorgas=='solv','SOLVORGAS NOT SET TO A VALID VALUE'
        
        if solvorgas=='gas':
            path=pathlib.Path(os.path.join(orig_dir, 'Calcs', str(i).zfill(7) , y, functional, basis,'gas', f'{str(i).zfill(7)}_{y}{sub_suff}.log'))
            
        elif solvorgas=='solv':
            path=pathlib.Path(os.path.join(orig_dir, 'Calcs',str(i).zfill(7) , y, functional, basis,solvent,solvmethod, f'{str(i).zfill(7)}_{y}{sub_suff}.log'))
        
        
        
        if exists(path): #If calculation was run
            
            if check(path, 'Normal termination of Gaussian',count=2): #If calculation was run successfully
                try:
                    
                    temp_data=cclib.io.ccread(str(path))

                    meta_data=temp_data.metadata
                    data[k,2*v]=temp_data.enthalpy
                    data[k,2*v+1]=temp_data.freeenergy
                    data[k,6]=checkvibs(temp_data.vibfreqs,asint=True)
                    temp_atom_coords.append(temp_data.atomcoords[-1,:,:])
                    temp_atom_labels.append([no_to_symbol(x) for x in temp_data.atomnos])
                    temp_wall_time.append(sum(meta_data['wall_time'][i].seconds for i in range(len(meta_data['wall_time']))))
                    
                    continue
                except Exception as e:
                    exc_type, exc_obj, exc_tb = sys.exc_info()
                    fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
                    print(e)       
                    print(exc_type, fname, exc_tb.tb_lineno)

                    data[k,2*v]= 1
                    data[k,2*v+1]= 1
                    data[k,6]=-1
                    temp_atom_coords.append([])
                    temp_atom_labels.append([])
                    temp_wall_time.append(0)
                    continue
            else: #Run unseccessfully
                
                #Check if geometry did not optimize
                if check(str(path),'Converged',120):
                    
                    temp_data=cclib.io.ccread(str(path))
                    meta_data=temp_data.metadata
                    data[k,2*v]=1
                    data[k,2*v+1]=1
                    data[k,6]=1
                    temp_atom_coords.append(temp_data.atomcoords[-1,:,:])
                    temp_atom_labels.append([no_to_symbol(x) for x in temp_data.atomnos])
                    temp_wall_time.append(0)
                    continue
                
                else:
                    data[k,2*v]= 1
                    data[k,2*v+1]= 1
                    data[k,6]=-1
                    temp_atom_coords.append([])
                    temp_atom_labels.append([])
                    temp_wall_time.append(0)
                    continue
        else:
            data[k,2*v]= 1
            data[k,2*v+1]= 1
            data[k,6]=-1
            temp_atom_coords.append([])
            temp_atom_labels.append([])
            temp_wall_time.append(0)
            continue

    atom_coords.append(temp_atom_coords)
    atom_labels.append(temp_atom_labels)
    wall_time.append(temp_wall_time)

100%|██████████| 1/1 [00:00<00:00,  3.94it/s]


In [28]:
tmp=combine_labels_and_xyz(atom_labels[2][0],atom_coords[2][0])
write_xyz_from_xyz_arr(tmp,'temp',include_total_atoms=True)


# Get Atom Labels & Coords

In [96]:
functional='M062X'
basis= 'Def2TZVP'
solvorgas='gas'
solvmethod='SMD'

orig_dir='/home/gridsan/groups/manthiram_lab/vHTP/Code/ylide/Library'
if solvorgas=='gas':
    solvent=='gas'
    ylide_type=[['ylide',''],['ylide_rad','_y_h_opt'],['ylide_h','']]
    #ylide_type=[['ylide_h','']]
else:
    solvent='acetonitrile'
    ylide_type=[['ylide','_gas_preopt'],['ylide_rad','_gas_preopt'],['ylide_h','_gas_preopt'] ]

start=0
stop=2
n=stop-start
data=np.empty((n,10))
atom_coords=[] #Dim1= Ylide Dim2=Ylide Rad Dim3= Ylide_H
atom_labels=[] #Dim1= Ylide Dim2=Ylide Rad Dim3= Ylide_H
wall_time=[] #Calculation time in seconds

for v,y_list in enumerate(ylide_type):
    
    y=y_list[0]
    sub_suff=y_list[1]
    
    temp_atom_coords=[]
    temp_atom_labels=[]
    temp_wall_time=[]
    
    for k,i in enumerate(tqdm(range(start,stop))):
        n_str=str(i).zfill(7)

        
        
        assert solvorgas == 'gas' or solvorgas=='solv','SOLVORGAS NOT SET TO A VALID VALUE'
        
        if solvorgas=='gas':
            path=pathlib.Path(os.path.join(orig_dir, 'Calcs', str(i).zfill(7) , y, functional, basis,'gas', f'{str(i).zfill(7)}_{y}{sub_suff}.xyz'))
            
        elif solvorgas=='solv':
            path=pathlib.Path(os.path.join(orig_dir, 'Calcs',str(i).zfill(7) , y, functional, basis,solvent,solvmethod, f'{str(i).zfill(7)}_{y}{sub_suff}.xyz'))
        
        
        
        if exists(path): #If calculation was run
            
            try:
                temp_temp_atom_labels=[]
                temp_temp_atom_coords=[]
                
                with open(path) as fp:
                    Lines = fp.readlines()
                    for line in Lines:
                        l=line.strip().split()
                        if len(l)>0:
                            temp_temp_atom_labels.append(l[0])
                            temp_temp_atom_coords.append(l[1:])
                temp_atom_labels.append(temp_temp_atom_labels)
                temp_atom_coords.append(np.array(temp_temp_atom_coords,dtype='float'))

                #temp_data=cclib.io.ccread(str(path))

#                     meta_data=temp_data.metadata
#                     data[k,2*v]=temp_data.enthalpy
#                     data[k,2*v+1]=temp_data.freeenergy
#                     data[k,6]=checkvibs(temp_data.vibfreqs,asint=True)
#                     temp_atom_coords.append(temp_data.atomcoords[-1,:,:])
                #temp_atom_labels.append([no_to_symbol(x) for x in temp_data.atomnos])
#                     temp_wall_time.append(sum(meta_data['wall_time'][i].seconds for i in range(len(meta_data['wall_time']))))
                continue
            except Exception as e:
                exc_type, exc_obj, exc_tb = sys.exc_info()
                fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
                print(e)       
                print(exc_type, fname, exc_tb.tb_lineno)

                data[k,2*v]= 1
                data[k,2*v+1]= 1
                data[k,6]=-1
                temp_atom_coords.append([])
                temp_atom_labels.append([])
                temp_wall_time.append(0)
                continue

        else:
            data[k,2*v]= 1
            data[k,2*v+1]= 1
            data[k,6]=-1
            temp_atom_coords.append([])
            temp_atom_labels.append([])
            temp_wall_time.append(0)
            continue

    atom_coords.append(temp_atom_coords)
    atom_labels.append(temp_atom_labels)
#     wall_time.append(temp_wall_time)

100%|██████████| 2/2 [00:00<00:00, 1314.21it/s]


In [97]:
atom_coords[0]

[array([[ 2.939539, -0.135209,  0.747927],
        [ 1.63468 , -0.036508, -0.004242],
        [ 1.61755 ,  0.053363, -1.22989 ],
        [ 0.603924, -0.064416,  0.854854],
        [-0.83583 ,  0.02553 ,  0.285024],
        [-1.038649, -1.167609, -0.613622],
        [-0.954057,  1.34706 , -0.43042 ],
        [-1.704374, -0.034578,  1.511128],
        [ 2.992552, -1.077977,  1.301215],
        [ 3.769893, -0.111218,  0.034963],
        [ 3.052782,  0.711862,  1.431589],
        [-2.102653, -1.209357, -0.868551],
        [-0.754258, -2.078542, -0.076667],
        [-0.464144, -1.057745, -1.536077],
        [-0.612027,  2.149589,  0.231366],
        [-0.383695,  1.333745, -1.361849],
        [-2.011635,  1.496307, -0.671436],
        [-1.530584, -0.98244 ,  2.031974],
        [-2.748865,  0.024012,  1.189201],
        [-1.470148,  0.814132,  2.16286 ]]),
 array([[-0.52282 ,  0.227167,  1.499165],
        [-0.502109, -0.138309,  0.035565],
        [-1.287269, -1.394155, -0.244137],
        [

In [70]:
with open(f"../pickled_data/atom_labels.pkl","wb") as file:
    pickle.dump(atom_labels, file)

In [19]:
np.mean(wall_time[2][:])

1876.582

#### Wall Times For Different Triples Conditons

[5,1,1] wall_time: [1816, 2098, 2524, 2864, 1405, 1256, 1628, 1559, 1741, 1783] <br>
[10,1,2] wall_time: [[1800, 0, 0, 0, 1398, 1248, 1618, 1560, 1732, 1787]] <br>
[5,2,1] wall_time: [3784, 0, 5337, 0, 0, 2829, 0, 3480, 3763, 0] <br>

There seems to be no effect from number of threads, it seems that we still have an effect from RAM and it scales pretty linearly so I will use 1 node per calculation

In [20]:
print('CALCULATING REDOX POTENTIALS')    
for i, v in enumerate(tqdm(range(np.shape(data)[0]))):
    data[i,7]=calcE(data[i,3],data[i,1])
    
    
print('CALCULATING DPFE')
for i, v in enumerate(tqdm(range(np.shape(data)[0]))):
    data[i,8]=calcDPFE(data[i,5],data[i,1],solvent=solvent)

print('CALCULATING GHBind ')
for i, v in enumerate(tqdm(range(np.shape(data)[0]))):
    data[i,9]=calcGHBind(data[i,5],data[i,3],solvent='gas')
    

100%|██████████| 1000/1000 [00:00<00:00, 703270.29it/s]

CALCULATING REDOX POTENTIALS
CALCULATING DPFE
CALCULATING GHBind 


In [12]:
!pwd

/home/gridsan/groups/manthiram_lab/vHTP/Code/ylide/Library


In [21]:
#Save Extracted data
cwd='/home/gridsan/jmaalouf/vHTP/Code/ylide/Library'
os.chdir(cwd)

with open(f"../pickled_data/atom_labels.pkl","wb") as file:
    pickle.dump(atom_labels, file)

with open(f"pickled_data/wall_time_{solvorgas}.pkl", "wb") as file: 
    pickle.dump(wall_time, file)

with open(f"pickled_data/atom_coords_{solvorgas}.pkl", "wb") as file: 
    pickle.dump(atom_coords, file)


np.save(f'pickled_data/data_{solvorgas}.npy', data)


In [58]:
df=pd.DataFrame(data, columns=['ylide_enthalpy','ylide_freeenergy','ylide_rad_enthalpy','ylide_rad_freeenergy','ylide_h_enthalpy','ylide_h_freeenergy','Vibs All Positivse','Redox Potential (V vs SHE)','Deprotonation Free Energy (kJ/mol)'])

In [43]:
#df.to_csv('ylide_calcs_6-311++G**.csv')